# Checking enviroment states and action sets

In [1]:
import sys
import os.path
sys.path.append("..")
from PPO_Agent_tf import *
from ENV_DETAILS import *

2023-12-11 14:14:27.686999: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Num devices available:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
if os.name == 'nt':
    main_hyper_dir = "D:\\Artificial_Intelligence\\Portfolio\\RL_updated\\Pendulum\\" # Windows
    conda_python_exec = 'C:\\Users\\yanie\\anaconda3\\envs\\ai_dev\\python.exe '# Windows
    logs_dir = main_hyper_dir+'Hyperparam_tuning\\' # Windows
else:
    main_hyper_dir = "/media/n/NewDisk/Artificial_Intelligence/Portfolio/RL_updated/Pendulum/" # Linux
    conda_python_exec = '/home/n/anaconda3/envs/ai_dev/bin/python '# Linux
    logs_dir = main_hyper_dir+'Hyperparam_tuning_a3c/' # Linux

ENV = "Pendulum-v1"
SUCESS_CRITERIA_VALUE = ENV_DETAILS[ENV]["SUCESS_CRITERIA_VALUE"]
SUCESS_CRITERIA_EPOCH = ENV_DETAILS[ENV]["SUCESS_CRITERIA_EPOCH"]
EPISODES = ENV_DETAILS[ENV]["EPISODES"]

In [3]:

env = gym.make(ENV)
env

<TimeLimit<OrderEnforcing<PendulumEnv<Pendulum-v1>>>>

In [4]:
env._max_episode_steps

200

In [5]:
s = env.observation_space.sample()
s

array([-0.09696076,  0.5834242 , -1.089912  ], dtype=float32)

In [6]:
env.observation_space, env.observation_space.shape[0]

(Box([-1. -1. -8.], [1. 1. 8.], (3,), float32), 3)

In [7]:
s = env.reset()
s[0]

-0.6571847

In [8]:
env.action_space

Box(-2.0, 2.0, (1,), float32)

In [9]:
s = env.reset()
env.action_space, env.action_space, env.step(s)

(Box(-2.0, 2.0, (1,), float32),
 Box(-2.0, 2.0, (1,), float32),
 (array([ 0.551725  , -0.8340261 , -0.09397238], dtype=float32),
  -0.9838920097072988,
  False,
  {}))

Testing keras model architecture 

In [10]:
print("Acquiring parameters ....")
writer= "Training/fit_ppo/"

entropy_factor = 0.05
discount = 0.99
dense_units_actor = [256,256]
num_layers_actor = 2
dense_units_critic = [256,256]
num_layers_crit =2

model = get_model( discount, dense_units_actor,  dense_units_critic, num_layers_actor, num_layers_crit, writer, 
                environment_name = ENV,reward_scaler = 1, lr_actor= 0.00001, lr_critic= 0.00001, 
                gae_lambda = 0.95, entropy_coeff = entropy_factor, policy_clip = 0.2, training_epoch = 50, memory_size= 50)
                 
obs = model.train_env.reset()
model.call([1, obs] )

Acquiring parameters ....


(array([ 0.4172543 , -0.90878975, -0.46718445], dtype=float32), False)

In [11]:
model([1, obs])

(array([ 0.36029434, -0.9328387 , -1.2367705 ], dtype=float32), False)

In [12]:
model = get_model( discount, dense_units_actor,  dense_units_critic, num_layers_actor, num_layers_crit, writer, 
                environment_name = ENV,reward_scaler = 1, lr_actor= 0.00001, lr_critic= 0.00001, 
                gae_lambda = 0.95, entropy_coeff = entropy_factor, policy_clip = 0.2, training_epoch = 50, memory_size= 50)

model.compile()
model.fit([-1], [-1], epochs= 10)

Trial number :  1
Epoch 1/10
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))
1/1 [==============================] - 1s 760ms/step - actor_loss: 18957.3047 - total_train_reward: -253.7482
Epoch 2/10
1/1 [==============================] - 1s 662ms/step - actor_loss: 17722.7832 - total_train_reward: -241.8282
Epoch 3/10
1/1 [==============================] - 1s 683ms/step - actor_loss: 34918.6445 - total_train_reward: -337.9176
Epoch 4/10
1/1 [==============================] - 1s 646ms/step - actor_loss: 37837.6641 - total_train_reward: -351.3461
Epoch 5/10
1/1 [==============================] - 1s 649ms/step - actor_loss: 57769.2852 - total_train_reward: -431.8658
Epoch 6/10
1/1 [==============================] - 1s 686ms/step - actor_loss: 18305.0195 - total_train_reward: -241.8495
Epoch 7/10
1/1 [======

### Training/Hyperparam run

In [13]:
%tensorboard --logdir logs_general/hyper

UsageError: Line magic function `%tensorboard` not found.


In [14]:
TUNING_TYPE = "BAYES"
HYPERPARAM_TUNING = True
writer= "Training/fit_PPO_tf/"

dir = r"Hyperparam_kt_ppo"
project_name = "keras_tunning_ppo_tf"

In [15]:
def model_builder(hp):
    

    max_layers_act = 3
    num_layers_crit= 3
    training_epoch = 50
    memory_size = env._max_episode_steps
    
    discount =  hp.Float('discount',0.85, 0.99, step =0.01)
    gae_lambda =  hp.Float('gae_lambda',0.85, 0.97, step =0.01)
    lr_actor = hp.Float('lr_actor', 0.000001, 0.005)
    lr_critic = hp.Float('lr_critic', 0.000001, 0.005)
    policy_clip = hp.Float('policy_clip', 0.1, 0.3, step =0.1)
    entropy_coeff = hp.Float('entropy_coeff', 0.001, 0.1)

    if max_layers_act>1 :
        num_layer_a = hp.Int('n_dense_layers_actor', 1, max_layers_act)
    else : 
        num_layer_a = max_layers_act
    dense_units_act =  [hp.Int('dense_units_act_'+str(i), 32, 512) for i in range(num_layer_a)]

    if num_layers_crit>1 :
        num_layer_c = hp.Int('n_dense_layers_critic', 1, num_layers_crit)
    else : 
        num_layer_c = num_layers_crit

    dense_units_crit =  [hp.Int('dense_units_crit_'+str(i), 32, 512) for i in range(num_layers_crit)]

    model = PPO(
        discount = discount, 
        dense_units_act = dense_units_act,
        dense_units_crit= dense_units_crit, 
        num_layer_act  = num_layer_a, 
        num_layer_crit= num_layer_c,
        writer = "logs_hyper/ppo_tf/",
        trial_n = get_valid_trials_number(dir+"/"+project_name ),
        evaluation_epoch = env._max_episode_steps,
        environment_name = ENV,
        reward_scaler = 1,
        lr_actor = lr_actor, lr_critic = lr_critic, 
        gae_lambda=gae_lambda,
        policy_clip = policy_clip,
        training_epoch = training_epoch,
        entropy_coeff = entropy_coeff,
        memory_size = memory_size
        )
    
    model.compile()

    return model

In [16]:
if HYPERPARAM_TUNING:


    tuner = kt.BayesianOptimization(
        model_builder,
        objective= kt.Objective('total_train_reward', direction="max"), 
        max_trials = 50,
        #distribution_strategy= strategy,
        directory=dir,
        project_name=project_name
        )
    tuner.search(x=[0], y=[1],  epochs= 10000)
else : 
    
        print("Acquiring parameters ....")
        writer= "Training/fit_ppo_tf/"

        training_steps = 1000000
        entropy_factor = 0.05
        discount = 0.99
        dense_units_actor = [128]#64, 32]
        num_layers_actor = 1
        dense_units_critic = [128]#64,32]
        num_layers_crit =1

        model = get_model(discount = discount, dense_units_act=dense_units_actor,  dense_units_crit=dense_units_critic, num_layer_a=num_layers_actor, num_layer_c = num_layers_crit,
                          writer = writer, environment_name = ENV,reward_scaler = 1,  lr_actor= 0.00001, lr_critic= 0.00001, 
                      gae_lambda = 0.95, entropy_coeff = entropy_factor, policy_clip = 0.2, training_epoch = 50)
                 
        model.compile()
        model.fit([-1], [-1], epochs= 1000000)


Trial 3 Complete [05h 04m 10s]
total_train_reward: -610.8777259528645

Best total_train_reward So Far: -610.740117344646
Total elapsed time: 14h 53m 40s

Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
0.98              |0.94              |discount
0.86              |0.95              |gae_lambda
0.0021516         |0.0018657         |lr_actor
0.00040776        |0.001466          |lr_critic
0.2               |0.1               |policy_clip
0.022371          |0.055969          |entropy_coeff
2                 |2                 |n_dense_layers_actor
159               |244               |dense_units_act_0
2                 |2                 |n_dense_layers_critic
484               |98                |dense_units_crit_0
218               |90                |dense_units_crit_1
457               |286               |dense_units_crit_2
133               |32                |dense_units_act_1
319               |None              |dense_units_act_2

Trial number : 

In [ ]:
exploration_tech = "soft"
hyperparam_combination=[]

for trials in tuner.oracle.get_best_trials(num_trials=50):
    if int(trials.trial_id) == 49:
        print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)

In [ ]:
val_env = gym.make(ENV, render_mode = "rgb_array")
dir = r"Hyperparam_kt_ppo"

for trials in tuner.oracle.get_best_trials(num_trials=1):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)

    training_steps = 1000000
    entropy_factor = trials.hyperparameters.values["entropy_coeff"]
    discount = trials.hyperparameters.values["discount"]
    dense_units_actor = [128]#64, 32]
    num_layers_actor = 1
    dense_units_critic = [128]#64,32]
    num_layers_crit =1

    model = run_training(training_steps,  discount, dense_units_actor,  dense_units_critic, num_layers_actor, num_layers_crit, writer, 
                    environment_name = ENV,reward_scaler = 1, return_agent = True, lr_actor= 0.00001, lr_critic= 0.00001, 
                    gae_lambda = 0.95, entropy_coeff = entropy_factor, policy_clip = 0.2, training_epoch = 50)

    
env_model = tuner.get_best_models()[0]
final_rewards = final_evaluation(env_model,val_env,n_tries=200, exploration=exploration_tech,  video_name = "./ppo_"+exploration_tech+"_video.mp4")
print("Final mean reward '",exploration_tech,"':", np.mean(final_rewards))